In [52]:
# handling BAG
import h5py
from io import BytesIO, StringIO
import xml.etree.ElementTree as ET
from xml import etree

import numpy as np


In [20]:
def getbagcrs(bagmetadata):
    #with help from Alex Leith
    
    
    tree = ET.parse('metadata.xml')
    root = tree.getroot()

    namespaces = {
        'gmd': "http://www.isotc211.org/2005/gmd",
        'gco': "http://www.isotc211.org/2005/gco"
    }

    referencesystem = root.find('gmd:referenceSystemInfo', namespaces)[0]

    referencesystemid = reference_system.find('gmd:referenceSystemIdentifier', namespaces).find('gmd:RS_Identifier', namespaces)

    projstring = reference_system_id.find('gmd:code', namespaces).find('gco:CharacterString', namespaces).text

    return projstring

In [21]:
bag = h5py.File('../../bag-samples/grid0_ellipsoid_1m.bag')

In [22]:
bag

<HDF5 file "grid0_ellipsoid_1m.bag" (mode r+)>

In [23]:
root = bag['BAG_root']
metadata = root['metadata']

In [79]:
metadata.value

array([b'<', b'?', b'x', ..., b'>', b'\n', b''], dtype='|S1')

In [83]:
metastr = BytesIO(metadata.value)

In [84]:
metastr

In [62]:
type(metastring)

bytes

In [70]:
mstring = metastring.decode()

In [72]:
type(mstring)

str

In [76]:
mstring = mstring.encode('utf-8')

In [77]:
parser = ET.XMLParser(encoding="utf-8")

In [85]:
ET.parse(mstring.decode())

ValueError: embedded null byte